In [1]:
from models import UNet, TransferSegmentation, save_model, load_model
import torch 
import torch.nn as nn 
from torchvision.transforms import v2 
from utils import load_data
import torch.utils.tensorboard as tb 
import numpy as np

torch.manual_seed(17)

DATASET_PATH = 'flood_area_dataset'

def calculate_iou(predicted_mask, target_mask, epsilon=1e-6):
    predicted_mask = torch.sigmoid(predicted_mask) # probabilidades

    predicted_mask = (predicted_mask > 0.5).float() # matriz true/false -> 1/0

    predicted_mask = predicted_mask.view(-1).bool() # permite hacer operacion & y |
    target_mask = target_mask.view(-1).bool()

    intersection = (predicted_mask & target_mask).float().sum()
    union = (predicted_mask | target_mask).float().sum()

    iou = (intersection+epsilon)/(union+epsilon)

    return iou.item()

In [2]:
import datetime 

train_logger = None 
log_dir = f'semantic_segmentation/runs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}' 

if log_dir:
    from os import path 
    train_logger = tb.SummaryWriter(path.join(log_dir), 'train', flush_secs=1)

In [3]:
'''model = UNet()'''
model = TransferSegmentation(n_classes=1)
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.CenterCrop(1000), # desde de la imagen, corto 1000 pixeles hacia afuera, padding=0
    v2.Resize((224, 224))
])
#transform = model.weights.transforms() # resulta mejor con la transformacion propia

train_loader = load_data(dataset_path=DATASET_PATH, transform=transform, batch_size=64)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

criterion = nn.BCEWithLogitsLoss() # aplica sigmoid directamente sobre los logits del modelo
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max')

In [4]:
# Training Loop
best_iou = 0
num_epochs = 100
i = 0
patience = 0
max_patience = 15

for epoch in range(num_epochs):
    running_loss = 0
    epoch_mean_iou = []

    for (inputs, targets) in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_iou = calculate_iou(outputs, targets)
        epoch_mean_iou.append(batch_iou)

        running_loss += loss.item()

        if log_dir is not None:
            train_logger.add_images('image', inputs[:6], i)
            train_logger.add_images('label', targets[:6], i)
            train_logger.add_images('pred', torch.sigmoid(outputs[:6])>0.5, i)
        
            if i==0:
                train_logger.add_graph(model, inputs) # representacion grafica del modelo
            i += 1
        
    # Epoch Estadisticas
    epoch_mean_iou = np.mean(epoch_mean_iou)*100
    epoch_loss = running_loss / len(train_loader)

    scheduler.step(epoch_mean_iou)

    if log_dir is not None:
        train_logger.add_scalar('epoch_mean_iou', epoch_mean_iou, epoch)
        train_logger.add_scalar('epoch_loss', epoch_loss, epoch)
        train_logger.add_scalar('epoch_lr', optimizer.param_groups[0]['lr'], epoch)
    
    print(f"Epoch {epoch+1}/{num_epochs} Loss: {epoch_loss:.2f} Mean IoU: {epoch_mean_iou:.2f}%")

    if epoch_mean_iou > best_iou:
        best_iou = epoch_mean_iou
        save_model(model, 'TransferSegmentation.th')
        patience = 0
    else:
        patience += 1
    
    if patience == max_patience:
        break


Epoch 1/100 Loss: 0.55 Mean IoU: 49.88%
Epoch 2/100 Loss: 0.41 Mean IoU: 63.52%
Epoch 3/100 Loss: 0.35 Mean IoU: 68.14%
Epoch 4/100 Loss: 0.32 Mean IoU: 71.93%
Epoch 5/100 Loss: 0.31 Mean IoU: 73.69%
Epoch 6/100 Loss: 0.30 Mean IoU: 76.00%
Epoch 7/100 Loss: 0.29 Mean IoU: 76.54%
Epoch 8/100 Loss: 0.28 Mean IoU: 78.11%
Epoch 9/100 Loss: 0.27 Mean IoU: 77.69%
Epoch 10/100 Loss: 0.27 Mean IoU: 77.72%
Epoch 11/100 Loss: 0.26 Mean IoU: 79.87%
Epoch 12/100 Loss: 0.25 Mean IoU: 80.90%
Epoch 13/100 Loss: 0.25 Mean IoU: 80.26%
Epoch 14/100 Loss: 0.25 Mean IoU: 80.97%
Epoch 15/100 Loss: 0.24 Mean IoU: 81.44%
Epoch 16/100 Loss: 0.24 Mean IoU: 81.95%
Epoch 17/100 Loss: 0.24 Mean IoU: 81.90%
Epoch 18/100 Loss: 0.23 Mean IoU: 83.07%
Epoch 19/100 Loss: 0.23 Mean IoU: 82.65%
Epoch 20/100 Loss: 0.23 Mean IoU: 83.30%
Epoch 21/100 Loss: 0.22 Mean IoU: 84.16%
Epoch 22/100 Loss: 0.22 Mean IoU: 83.78%
Epoch 23/100 Loss: 0.22 Mean IoU: 84.27%
Epoch 24/100 Loss: 0.22 Mean IoU: 84.91%
Epoch 25/100 Loss: 0.21 M